## Credit Risk Prediction — Model Training
### Objective
This notebook trains and evaluates multiple machine learning models
to predict **credit default risk** using the processed dataset.

Models covered:
- Logistic Regression (baseline credit scorecard)
- Random Forest (non-linear ensemble)
- XGBoost (advanced boosting model)

Evaluation focuses on:
- ROC-AUC (primary metric)
- Confusion Matrix
- Precision & Recall
